<a href="https://colab.research.google.com/github/azzbc7819/test_igaimers/blob/develop/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B2%98%EB%A6%AC_withPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader


In [4]:
import numpy as np

In [5]:
#데이터 전처리

trainfile = '/content/drive/MyDrive/LG Aimers 해커톤/train.csv'
train_df = pd.read_csv(trainfile)

testfile = '/content/drive/MyDrive/LG Aimers 해커톤/test.csv'
test_df = pd.read_csv(trainfile)

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP','Y_Class', 'Y_Quality']) 
train_y = pd.DataFrame(train_df['Y_Class']) 

test_x = test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

#결측값 모두 0으로 채우기
train_x = train_x.fillna(0) 
test_x = test_x.fillna(0)

#말그대로 범주형 데이터 > 수치형 데이터로 전환

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]): #label == test 데이터의 범주형 데이터. np.unique()고유한 값만 모아서 반환 반복X
        if label not in le.classes_: #train데이터에서 확인된 범주형 데이터들에 포함되지 않는 경우
            le.classes_ = np.append(le.classes_, label) #해당 데이터도 추가
    test_x[i] = le.transform(test_x[i])  
    
print('Done.')

Done.


In [6]:
class CustomDataset(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = torch.Tensor(x_data)
    self.y_data = torch.Tensor(y_data)

  def __len__(self):
    # 가지고 있는 데이터셋의 길이를 반환한다.
    return len(self.x_data)

  def __getitem__(self,idx):
    return self.x_data[idx], self.y_data[idx] # 해당하는 idx(인덱스)의 x와 y 데이터를 반환한다.

In [7]:
print(type(train_x.values))
print(type(train_x))

#그냥 data는 pd.df라서 torch에서 못 읽음 values로 해주어야 함

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


In [9]:
#DataLoader을 통해 DataSet으로 전처리하 데이터셋의 샘플들을 미니배치로 전달하고, 매 에폭마다 데이터를 다시 섞어준다
dataset = CustomDataset(train_x.values, train_y.values)
dataloader = DataLoader(dataset, batch_size = 60, shuffle=True)

In [10]:
trbatch_x, trbatch_y = next(iter(dataloader))

In [11]:
#batchsize, feature 수
print(trbatch_x.size())
print(trbatch_y.size())

torch.Size([60, 2877])
torch.Size([60, 1])


In [13]:
print(trbatch_x[0])

tensor([4., 2., 2.,  ..., 0., 0., 0.])


In [15]:
trbatch_x, trbatch_y = next(iter(dataloader))

In [16]:
print(trbatch_x[0])

#위랑 달라진 걸 볼 수 있음 next했으니까 다음 배치로 넘어간 것

tensor([3., 0., 0.,  ..., 0., 0., 0.])
